In [2]:
# Desenvolvido em python (jupyter)

#
# Neste código é plotado o mapa mundi. Em especial foram plotadas bolinhas representando cada condado dos EUA. 
# A cor de cada bolinha está relacionada com o valor do sinal referente ao condado. No caso em questão
# tem-se que o sinal varia de 0 até 1890, a relação entre o valor do sinal e a cor está ilustrada na barra de cores 
# acima do gráfico.
#


# Para que o código funcione é precio instalar as bibliotecas abaixo. 
# "folium" e "branca.colormap" são responsáveis pela plotagem do mapa 

import folium
import csv
import numpy as np
import branca.colormap as cm
from collections import defaultdict



# Código para abrir o arquivo (baixado diretamente do datausa.io) e armazenar os valores numa matriz de numpy
# para processar os dados. Cada coluna da matriz se refere a uma coluna do arquivo 'csv' 

columns = defaultdict(list) 

with open('violent_crimes.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        for (i,v) in enumerate(row):
            columns[i].append(v)
            
# Em column[0] tem-se, por exemplo, o nome de todos os condados

# "l" recebe o comprimento da coluna
l = len(columns[0])

# Faz-se o mesmo, mas agora para um arquivo contendo as latitudes e longitudes dos condados

ccolumns = defaultdict(list) 

with open('fips_lat_long.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        for (i,v) in enumerate(row):
            ccolumns[i].append(v)
            

k = len(ccolumns[0])

# Agora cria-se uma matriz que vai armazenar a identificação, latitude, longitude e sinal para cada condado

z = 0
B = np.zeros((l,4))

for i in range (0,l):
    
    a = columns[2][i]
    b = columns[3][i]
    a = a[-5:]
    
    
    for j in range (0,k):
        
        if (a == ccolumns[0][j] and float(ccolumns[2][j]) < -64.0 and float(ccolumns[2][j]) > -127.0):
            
            try:
                B[z][0] = ccolumns[0][j]
                B[z][1] = ccolumns[1][j]
                B[z][2] = ccolumns[2][j]
                B[z][3] = float(columns[3][i])
            except ValueError:
                print (i)        
                break

            z = z + 1

B = B[~np.all(B == 0, axis=1)]

# B[][0] contém o FIPS, B[][1] contém a latitude, B[][2] a longitude e B[][3] o sinal
    
# É preciso criar uma matriz com duas colunas para as coordenadas e um vetor coluna para o sinal

coords2 = np.zeros((len(B),2))
s = np.zeros((len(B),1))

for p in range(0,len(B)):
    coords2[p][0] = B[p][1]
    coords2[p][1] = B[p][2]
    s[p] = B[p][3]
    

# Coordenadas iniciais do mapa
COORD = [32.536382 ,  -86.64449]


# Plota-se agora o mapa
# É possível alterar o estilo visual do mapa através do parâmetro 'tiles'. 

map_nyc = folium.Map(location=COORD, zoom_start=3, tiles='cartodbpositron', width=640, height=480)

# 
coords2 = list(coords2)

# Cria-se um objeto (aux) para obter as cores da barra (RdBu_04) e as cores são armazenadas em uma lista (color = [])

color = []

aux = cm.linear.RdBu_04.scale(0, 1890)

for i in range(0,len(coords2)):
    color.append(aux.rgb_hex_str(s[i]))

# Agora são plotadas as bolinhas
for i in range(0,len(coords2)):
    folium.CircleMarker(coords2[i], radius=2, fill_opacity=1, color=color[i], fill_color=color[i]).add_to(map_nyc) 

# Plota-se agora a barra de cores
colormap = cm.linear.RdBu_04.scale(0, 1890)
colormap.caption = 'A colormap caption'
map_nyc.add_child(colormap)


# Display map in Jupyter
map_nyc

